In [1]:
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.detection.config import Config as DetConfig
from speed_trapv3.keypoints.config import Config as KeyConfig
from speed_trapv3.tracking.config import Config as TrackConfig
from speed_trapv3.config import Config
from speed_trapv3.utils import slugify, get_prediction_path
from pathlib import Path
import random
from tqdm import tqdm
import imageio
from speed_trapv3.keypoints.dataset import crop_and_resize,process_keypoints, keypoints_post_inference_processing
import numpy as np
import torchvision.transforms as T
import torch
import cv2
import os
import copy
import json
from speed_trapv3.tracking.tracking import get_video_properties, transform_image, write_to_json#, get_frame_box
from sparrow_tracky import Tracker, euclidean_distance, compute_moda_by_class, compute_moda
from sparrow_datums import AugmentedBoxTracking, BoxTracking, FrameBoxes, PType
from operator import itemgetter

/usr/local/lib/python3.9/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from speed_trapv3.detection.config import Config
from speed_trapv3.detection.dataset import RetinaNetDataset
from speed_trapv3.detection.model import RetinaNet
from speed_trapv3.utils import Holdout, batch_moda, result_to_boxes

In [3]:
detection_model = RetinaNet().eval().cuda()
detection_model.load(DetConfig.trained_model_path)

In [4]:
test_dataset = RetinaNetDataset(Holdout.TEST)

In [5]:
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)

In [6]:
for batch in iter(test_dataloader):
    x = batch['image']
print(batch.keys())

dict_keys(['image', 'boxes', 'labels'])


In [7]:
moda = 0
count = 0
for batch in iter(test_dataloader):
    x = batch['image']
    x = x.cuda()
    sample = {'boxes':batch['boxes'][0], 'labels':batch['labels'][0]}
    result = detection_model(x)[0]
    predicted_boxes = result_to_boxes(result)
    predicted_boxes = predicted_boxes[predicted_boxes.scores > DetConfig.score_threshold]
    ground_truth_boxes = result_to_boxes(sample)
    #Enable this line to calculate classwise.
    # moda_dict = compute_moda_by_class(predicted_boxes, ground_truth_boxes)
    # moda += sum(moda_dict.values())
    frame_moda = compute_moda(predicted_boxes, ground_truth_boxes)
    moda += frame_moda.value
    count += 1

/usr/local/lib/python3.9/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
print(f"Based on {count} test examples, the Multiple Object Detection Accuracy is {100 * moda/count}%.")
print(f"That means that the detection model can draw a bounding box around every vehicle with {100 * moda/count}% accuracy.")

Based on 43 test examples, the Multiple Object Detection Accuracy is 98.06201550387595%.
That means that the detection model can draw a bounding box around every vehicle with 98.06201550387595% accuracy.
